<a href="https://colab.research.google.com/github/Tlen33/Text-Segmentation/blob/main/%D0%A1%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8F_%D0%A2%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pymorphy2

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 6.8 MB/s 


In [ ]:
!pip install tensorflow==2.3

     |████████████████████████████████| 320.4 MB 47 kB/s 
     |████████████████████████████████| 20.1 MB 35.3 MB/s 
     |████████████████████████████████| 2.9 MB 48.9 MB/s 
     |████████████████████████████████| 459 kB 49.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:


In [ ]:
# Подключаем библиотеки

import numpy as np
import tensorflow
import pymorphy2
import re
import os
import pandas as pd
import time
import nltk
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras import utils
from google.colab import files
from gensim.models import word2vec
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Подключаем диск

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Считываем текст из файла и удаляем ненужные нам знаки

def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%']

  for dS in delSymbols:
    text = text.replace(dS, " ")

  text = re.sub("[.]", " ", text)
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  text = ' '.join(text.split()) 

  text = text.lower()
  return text

def text2Words(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split(' ')
  docs = [morph.parse(word)[0].normal_form for word in words]
  return docs

In [ ]:
# Подключаем директорию с договорами

directory = '/content/drive/MyDrive/Colab Notebooks/Базы/Договора432/'
os.listdir(directory)[250:255]

['dogovor-zaloga-privatizirovannoy-kvartiri.txt',
 'dogovor-zaloga-s-peredachej-predmeta-zaloga-zalogoderzhatelyu.txt',
 'dogovor-zaloga-nedvizhimosti.txt',
 'dogovor-zaloga-imuschestvennih-prav.txt',
 'dogovor-zaloga-imuschestva-kvartiri.txt']

In [ ]:
print('Всего', len(os.listdir(directory)), 'договора')

Всего 432 договора


In [ ]:
curTime = time.time()
agreements = []
for filename in os.listdir(directory):
  txt = readText(directory + filename)
  if txt != '':
    agreements.append(readText(directory + filename))
print('Загрузка файла заняла: ', round(time.time() - curTime, 2), 'с.')

Загрузка файла заняла:  7.45 с.


In [ ]:
# Пример договора

n = 7
print(os.listdir(directory)[n])
agreements[n]

3 (2).txt


'договор благотворительного пожертвования некоммерческой организации г г в лице действующего на основании именуемый в дальнейшем благотворитель с одной стороны и в лице действующего на основании именуемый в дальнейшем организация с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 общие положения 1 1 является некоммерческой организацией созданной в соответствии с законодательством российской федерации 1 2 организация в соответствии с уставом преследует цели 1 3 благотворитель является <s6> ведущей свою деятельность на территории </s6> 1 4 целью настоящего договора является взаимодействие сторон настоящего договора по осуществлению благотворителем своих пожертвований 1 5 на настоящий договор распространяется действие ст 582 гражданского кодекса российской федерации 2 предмет договора 2 1 <s1> <s3> благотворитель безвозмездно передает организации денежные средства далее пожертвования на цели указанные в п 2 2 настоящего догов

In [ ]:
# Переводим договор в лист слов

docs_full = []
curTime = time.time()
for i in range(len(agreements)):
  docs_full.append(text2Words(agreements[i]))
print('Превращение заняло: ', round(time.time() - curTime, 2), 's.')

Превращение заняло:  194.82 s.


In [ ]:
# Выводим число записей в наборе данных

docs = docs_full[0:-10]
docsToTest = docs_full[-10:]
print(len(docs_full))
print('Число текстов в для тестовой проверки в конце ноутбука:', len(docsToTest)) 
print('Число договоров для обучающей и проверочной выборках:',len(docs)) 
print('Число слов в первом договоре:', len(docs[10]))
print('Пример текста обычного:')
print(agreements[4][:62], '\n')
print('Тот же текст, но представленный ввиде лемм:')
print(docs[4][:10])

422
Число текстов в для тестовой проверки в конце ноутбука: 10
Число договоров для обучающей и проверочной выборках: 412
Число слов в первом договоре: 502
Пример текста обычного:
договор бартера г г в лице действующего на основании именуемый 

Тот же текст, но представленный ввиде лемм:
['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать']


In [ ]:
# Применям токинайзер для формирования xTrain

tokenizer = Tokenizer(lower=True, filters = '', char_level=False)
tokenizer.fit_on_texts(docs_full)
clean_voc = {}

for item in tokenizer.word_index.items():
  clean_voc[item[0]] = item[1]

print('Словарь все слов по их частотноти:') 
print(clean_voc, '\n')
print('Длина словаря:', len(clean_voc))
tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь все слов по их частотноти:
{'в': 1, 'договор': 2, 'и': 3, '2': 4, '1': 5, 'по': 6, 'сторона': 7, 'с': 8, 'настоящий': 9, 'на': 10, '3': 11, '<s1>': 12, '</s1>': 13, 'не': 14, '4': 15, 'или': 16, 'за': 17, 'он': 18, 'случай': 19, 'о': 20, '5': 21, '<s3>': 22, '</s3>': 23, 'право': 24, 'работа': 25, 'срок': 26, '<s4>': 27, '</s4>': 28, 'от': 29, 'при': 30, 'они': 31, '6': 32, 'другой': 33, 'быть': 34, 'лицо': 35, 'имущество': 36, 'день': 37, 'действовать': 38, 'арендатор': 39, 'условие': 40, 'для': 41, '7': 42, 'к': 43, 'если': 44, 'п': 45, 'дальнейший': 46, 'работник': 47, 'который': 48, 'а': 49, 'обязательство': 50, 'страховой': 51, 'законодательство': 52, 'порядок': 53, 'сумма': 54, 'размер': 55, 'действие': 56, 'арендодатель': 57, 'свой': 58, 'счёт': 59, 'соответствие': 60, 'указанный': 61, 'именовать': 62, 'мочь': 63, 'из': 64, 'основание': 65, 'обязанность': 66, 'обязать': 67, 'исполнение': 68, 'один': 69, 'страховщик': 70, 'страхователь': 71, 'иной': 72, 'год': 73, 'до': 7

In [ ]:
# Превращаем текст в последовательность индексов согласно словарю частотности

tok_agreem = tokenizer.texts_to_sequences(docs)
print("Взгляните на фрагмент обучающей выборки:")
print("Исходный текст:              ", docs[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreem[4][:20], '\n')

Взгляните на фрагмент обучающей выборки:
Исходный текст:               ['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'сторона', '1', 'с', 'один', 'сторона', 'и', 'в', 'лицо']
Тот же текст, но как последовательность индексов:  [2, 4233, 76, 76, 1, 35, 38, 10, 65, 62, 1, 46, 7, 5, 8, 69, 7, 3, 1, 35] 



In [ ]:
# Формируем yTrain

def getXYSamples(tok_agreem, tags_index):
  tags01 = []
  indices = []
 
  for agreement in tok_agreem:
    tag_place = [0, 0, 0, 0, 0, 0]
    for ex in agreement:
        if ex in tags_index:
          place = np.argwhere(tags_index==ex)
          if len(place)!=0:
            if place[0][0]<6:
              tag_place[place[0][0]] = 1
            else: 
              tag_place[place[0][0] - 6] = 0
        else:          
          tags01.append(tag_place.copy())
          indices.append(ex)

  return indices, tags01

def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items()))
  docs = [reverse_word_map.get(letter) for letter in x]

  return docs # Возвращает текст

In [ ]:
# Переводим теги в индексы

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)]
closetags = ['</s' + str(i) + '>' for i in range(1, 7)]
tags_index.extend(closetags)
tags_index = np.array([clean_voc[i] for i in tags_index])
print('Индексы всех тегов:', tags_index)
curTime = time.time()
xData, yData = getXYSamples(tok_agreem,tags_index)
decoded_text = reverseIndex(clean_voc, xData)
print('Превращение заняло: ', round(time.time() - curTime, 2), 'с.')

Индексы всех тегов: [ 12 341  22  27 117 278  13 352  23  28 118 273]
Превращение заняло:  2.98 с.


In [ ]:
# Выводим полученный результат

print('длина xData:', len(xData))
print('длина yData:', len(yData))
print('Посмотрим на исходные слова: ', docs[0][50:80])
print('Сдекодированые слова:', decoded_text[50:80])
print('Часть xData:     ', xData[50:80])
print('Часть yData:     ', yData[50:80])

длина xData: 408682
длина yData: 408682
Посмотрим на исходные слова:  ['1', 'предмет', 'договор', '1', '1', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по']
Сдекодированые слова: ['1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор']
Часть xData:      [5, 112, 2, 5, 5, 6, 9, 2, 224, 115, 918, 319, 125, 18, 250, 3, 546, 244, 319, 1, 396, 5, 4, 250, 304, 10, 162, 6, 9, 2]
Часть yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0,

In [ ]:
# Создание выборки из индексов

def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = []
  wordsLen = len(wordIndices)
  index = 0
  
  while (index + xLen <= wordsLen):
    xBatch.append(wordIndices[index:index+xLen])
    index += step

  return xBatch

In [ ]:
# Генерируем выборки с параметрами наших окон

xLen = 256
step = 30
embeddingSize = 300
xTrain = getSetFromIndices(decoded_text, xLen, step)
yTrain = getSetFromIndices(yData, xLen, step)

In [ ]:
# Выводим полученный результат

print('Длина xTrain:', len(xTrain))
print('Длина yTrain:', len(yTrain))
print('Длина примера из xTrain:',len(xTrain[0]))
print('Длина примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')
print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длина xTrain: 13615
Длина yTrain: 13615
Длина примера из xTrain: 256
Длина примера из yTrain: 256 

Пример xTrain ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'хранитель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'поклажедатель', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь', '1', '3', 'при', 'передача', 'вещь', 'по

In [ ]:
# Превращаем текст в последовательность индексов

tok_agreemTest = tokenizer.texts_to_sequences(docsToTest)

print("Посмотрим на фрагмент тестового текста:")
print("Исходный текст:              ", docsToTest[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[4][:20], '\n')

xDataTest, yDataTest = getXYSamples(tok_agreemTest,tags_index)
decoded_text = reverseIndex(clean_voc, xDataTest)

print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))
print('Сдекодированные текст:', decoded_text[50:80])
print('Часть xDataTest:     ', xDataTest[50:80])
print('Часть yDataTest:     ', yDataTest[50:80])

xLen = 256
step = 30
embeddingSize = 300

xTest = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном Xlen слов
yTest = getSetFromIndices(yDataTest, xLen, step) # # Последовательность индексов с окном Xlen от тегов

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера xTest:',len(xTest[0]))
print('Длина примера yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')
print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Посмотрим на фрагмент тестового текста:
Исходный текст:               ['трудовой', 'контракт', 'с', 'специалист', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'работодатель', 'с', 'один', 'сторона', 'и', 'гражданин']
Тот же текст, но как последовательность индексов:  [79, 187, 8, 897, 76, 76, 1, 35, 38, 10, 65, 62, 1, 46, 129, 8, 69, 7, 3, 265] 

Длина xDataTest: 12768
Длина yDataTest: 12768
Сдекодированные текст: ['1', 'предмет', 'договор', '1', '1', 'работник', 'приниматься', 'на', 'работа', 'на', 'предприятие', 'на', 'должность', 'руководитель', 'служба', 'реклама', 'и', 'маркетинг', 'для', 'выполнение', 'трудовой', 'обязанность', 'связанный', 'с', 'руководство', 'указанный', 'служба', 'планирование', 'и', 'стимулирование']
Часть xDataTest:      [5, 112, 2, 5, 5, 47, 958, 10, 25, 10, 228, 10, 495, 675, 640, 1157, 3, 1899, 41, 96, 79, 66, 286, 8, 889, 61, 640, 2325, 3, 3563]
Часть yDataTest:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0,

In [ ]:
# Функция ошибки

def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [ ]:
# Функция сети UNET - добавил Блок 5 и UP 1

def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):
    img_input = Input(input_shape)

    # Блок 1
    x = Conv1D(64 * k , 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_1_out = Activation('relu')(x)

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(128 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_4_out = Activation('relu')(x)

    # Блок 5
    x = MaxPooling1D()(block_4_out)
    x = Conv1D(1024 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(1024 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(1024 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_5_out = Activation('relu')(x)
    x = block_5_out

    # UP 1
    x = Conv1DTranspose(512 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_4_out])
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_3_out])
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x)
    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), loss='categorical_crossentropy', metrics=[dice_coef])

    return model

In [ ]:
# Создание xTrain и yTrain используя Word2Vec GENSIM

def pad_zeros(phrase, xLen = 256):
  while len(phrase) < xLen:
    phrase.append([0] * embeddingSize)
  if len(phrase) > xLen:
    phrase = phrase[:xLen]
  return phrase

def getSets(model, senI, tagI):
  xVector = []
  tmp = []
  for text in senI:
    tmp=[]
    for word in text:
      try:
        tmp.append(model[word])
      except:
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
curTime = time.time()

modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print('Обучение GENSIM модели заняло: ', round(time.time() - curTime, 2), 'с.')

Обучение GENSIM модели заняло:  43.15 с.


In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (13615, 256, 300)
Размерности yTrain: (13615, 256, 6)


In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_56 (Conv1D)              (None, 256, 64)      57664       input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_61 (BatchNo (None, 256, 64)      256         conv1d_56[0][0]                  
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 256, 64)      0           batch_normalization_61[0][0]     
_______________________________________________________________________________________

In [ ]:
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=15, batch_size=64)

Epoch 1/15
213/213 [==============================] - 15s 72ms/step - loss: 0.5426 - dice_coef: 0.5389 - val_loss: 3.4285 - val_dice_coef: 0.1776
Epoch 2/15
213/213 [==============================] - 14s 66ms/step - loss: 0.3374 - dice_coef: 0.6453 - val_loss: 1.1542 - val_dice_coef: 0.4661
Epoch 3/15
213/213 [==============================] - 14s 67ms/step - loss: 0.2846 - dice_coef: 0.6825 - val_loss: 0.8648 - val_dice_coef: 0.5526
Epoch 4/15
213/213 [==============================] - 14s 67ms/step - loss: 0.2593 - dice_coef: 0.7047 - val_loss: 0.9541 - val_dice_coef: 0.5392
Epoch 5/15
213/213 [==============================] - 14s 66ms/step - loss: 0.2441 - dice_coef: 0.7194 - val_loss: 1.0322 - val_dice_coef: 0.5263
Epoch 6/15
213/213 [==============================] - 14s 66ms/step - loss: 0.2410 - dice_coef: 0.7242 - val_loss: 0.9797 - val_dice_coef: 0.5524
Epoch 7/15
213/213 [==============================] - 14s 66ms/step - loss: 0.2313 - dice_coef: 0.7349 - val_loss: 1.0998 - 

In [ ]:
# Проверяем работу сети

def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 6)
  incorrect_list =  np.array([0] * 6)
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1

  for element in range(YY_array.shape[0]):
    for word in range(YY_array.shape[1]):
      for category in range(YY_array.shape[2]):
        if pred[element][word][category] == YY_array[element][word][category]:
          correct_list[category] += 1
        else:
          incorrect_list[category] += 1
      
  for i in range(6):
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [ ]:
print('Размер xTrain:', GENSIMtestX.shape)
print('Размер yTrain:', GENSIMtestY.shape)

Размер xTrain: (418, 256, 300)
Размер yTrain: (418, 256, 6)


In [ ]:
# Проверяем работу UNET

tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1)

Сеть распознала категорию  'S1' с точностью в 75.51%
Сеть распознала категорию  'S2' с точностью в 96.45%
Сеть распознала категорию  'S3' с точностью в 77.46%
Сеть распознала категорию  'S4' с точностью в 73.73%
Сеть распознала категорию  'S5' с точностью в 91.04%
Сеть распознала категорию  'S6' с точностью в 97.32%
Средняя точность 85.25%


In [ ]:
# Функция сети UNET № 2 - увеличил количество нейронов

def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):
    img_input = Input(input_shape)

    # Блок 1
    x = Conv1D(128 * k , 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(128 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_1_out = Activation('relu')(x)

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_4_out = Activation('relu')(x)

    # Блок 5
    x = MaxPooling1D()(block_4_out)
    x = Conv1D(1024 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(1024 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(1024 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_5_out = Activation('relu')(x)
    x = block_5_out

    # UP 1
    x = Conv1DTranspose(512 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_4_out])
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_3_out])
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_2_out])
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_1_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x)
    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), loss='categorical_crossentropy', metrics=[dice_coef])

    return model

In [ ]:
# Обучени сети UNET № 2 - увеличил количество нейронов, увеличил количество эпох до 50

model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=50, batch_size=64)

Epoch 1/50
213/213 [==============================] - 18s 85ms/step - loss: 0.5161 - dice_coef: 0.5493 - val_loss: 1.9699 - val_dice_coef: 0.4022
Epoch 2/50
213/213 [==============================] - 17s 81ms/step - loss: 0.3161 - dice_coef: 0.6594 - val_loss: 3.1254 - val_dice_coef: 0.3095
Epoch 3/50
213/213 [==============================] - 17s 81ms/step - loss: 0.2646 - dice_coef: 0.6976 - val_loss: 0.9818 - val_dice_coef: 0.4457
Epoch 4/50
213/213 [==============================] - 17s 81ms/step - loss: 0.2461 - dice_coef: 0.7144 - val_loss: 0.8993 - val_dice_coef: 0.5684
Epoch 5/50
213/213 [==============================] - 17s 81ms/step - loss: 0.2280 - dice_coef: 0.7318 - val_loss: 1.2268 - val_dice_coef: 0.4772
Epoch 6/50
213/213 [==============================] - 17s 81ms/step - loss: 0.2328 - dice_coef: 0.7300 - val_loss: 0.9098 - val_dice_coef: 0.5977
Epoch 7/50
213/213 [==============================] - 17s 81ms/step - loss: 0.2148 - dice_coef: 0.7495 - val_loss: 1.0785 - 

In [ ]:
# Проверяем работу UNET № 2

tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1)

Сеть распознала категорию  'S1' с точностью в 68.73%
Сеть распознала категорию  'S2' с точностью в 96.9%
Сеть распознала категорию  'S3' с точностью в 80.57%
Сеть распознала категорию  'S4' с точностью в 88.92%
Сеть распознала категорию  'S5' с точностью в 93.54%
Сеть распознала категорию  'S6' с точностью в 98.57%
Средняя точность 87.87%


In [ ]:
# Функция сети UNET № 3 - увеличил количество нейронов

def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):
    img_input = Input(input_shape)

    # Блок 1
    x = Conv1D(256 * k , 3, padding='same')(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_1_out = Activation('relu')(x)

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_4_out = Activation('relu')(x)

    # Блок 5
    x = MaxPooling1D()(block_4_out)
    x = Conv1D(1024 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(1024 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(1024 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_5_out = Activation('relu')(x)
    x = block_5_out

    # UP 1
    x = Conv1DTranspose(512 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_4_out])
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 2
    x = Conv1DTranspose(512 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_3_out])
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 3
    x = Conv1DTranspose(512 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_2_out])
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(512 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = concatenate([x, block_1_out])
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x)
    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), loss='categorical_crossentropy', metrics=[dice_coef])

    return model

In [ ]:
# Обучени сети UNET № 3 - увеличил количество нейронов, увеличил количество эпох до 100

model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=100, batch_size=64)

Epoch 1/100
213/213 [==============================] - 37s 175ms/step - loss: 0.6074 - dice_coef: 0.5201 - val_loss: 0.7458 - val_dice_coef: 0.4612
Epoch 2/100
213/213 [==============================] - 36s 170ms/step - loss: 0.4068 - dice_coef: 0.6096 - val_loss: 1.4740 - val_dice_coef: 0.3934
Epoch 3/100
213/213 [==============================] - 36s 170ms/step - loss: 0.3013 - dice_coef: 0.6726 - val_loss: 1.0344 - val_dice_coef: 0.4193
Epoch 4/100
213/213 [==============================] - 36s 170ms/step - loss: 0.2508 - dice_coef: 0.7091 - val_loss: 2.1090 - val_dice_coef: 0.4165
Epoch 5/100
213/213 [==============================] - 36s 170ms/step - loss: 0.2478 - dice_coef: 0.7152 - val_loss: 0.8437 - val_dice_coef: 0.5431
Epoch 6/100
213/213 [==============================] - 36s 170ms/step - loss: 0.2272 - dice_coef: 0.7313 - val_loss: 1.1649 - val_dice_coef: 0.5631
Epoch 7/100
213/213 [==============================] - 36s 169ms/step - loss: 0.2208 - dice_coef: 0.7401 - val_l

In [ ]:
# Проверяем работу UNET № 3

tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1)

Сеть распознала категорию  'S1' с точностью в 77.94%
Сеть распознала категорию  'S2' с точностью в 97.97%
Сеть распознала категорию  'S3' с точностью в 63.13%
Сеть распознала категорию  'S4' с точностью в 86.22%
Сеть распознала категорию  'S5' с точностью в 92.19%
Сеть распознала категорию  'S6' с точностью в 99.3%
Средняя точность 86.13%
